<a href="https://colab.research.google.com/github/Amir-Fasil/Prompt_Vs_finetuning/blob/main/Prompt_Vs_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing Requierments**

In [ ]:
!pip install langchain-community langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00


In [ ]:
!pip install -q transformers datasets peft accelerate torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import (
    GPT2Tokenizer,
    GPT2ForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import Dataset
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
import os
import kagglehub
from sklearn.model_selection import train_test_split

**Data Loading and Preparation**

In [ ]:
def load_prep():

    path = kagglehub.dataset_download("waalbannyantudre/hate-speech-detection-curated-dataset")
    df = pd.read_csv(f"{path}/HateSpeechDatasetBalanced.csv")

    # preparation of data

    df_label_1 = df[df['Label'] == 1].sample(n=10000, random_state=42)
    df_label_0 = df[df['Label'] == 0].sample(n=10000, random_state=42)

    df_balanced = pd.concat([df_label_1, df_label_0]).sample(frac=1, random_state=42).reset_index(drop=True)

    train_df, test_df = train_test_split(df_balanced, test_size=0.2, stratify=df_balanced['Label'], random_state=42)

    # Convert to Hugging Face Dataset
    train_dataset = Dataset.from_pandas(train_df)
    eval_dataset = Dataset.from_pandas(test_df)

    # Reduce size for Colab (optional)
    train_dataset = train_dataset.select(range(min(2000, len(train_dataset))))
    eval_dataset = eval_dataset.select(range(min(500, len(eval_dataset))))

    # Rename label column to 'labels'
    train_dataset = train_dataset.rename_column("Label", "labels")
    eval_dataset = eval_dataset.rename_column("Label", "labels")

    return train_dataset, eval_dataset

train_dataset, eval_dataset = load_prep()

In [ ]:
train_dataset

Dataset({
    features: ['Content', 'labels', '__index_level_0__'],
    num_rows: 2000
})

**Load GPT2-Large model and Tokenizer**

In [ ]:
def load_model_tokenizer():

    model_name = "gpt2-large"
    tokenizer = GPT2Tokenizer.from_pretrained(model_name) # fetching tokenizer for hf that fitted the model
    # for more datapreparation we need to make sure the text size is equal so we use both truncation and padding
    tokenizer.pad_token = tokenizer.eos_token

    model = GPT2ForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,  # Binary classification
    device_map = "auto",
    pad_token_id=tokenizer.eos_token_id

    )

    return model, tokenizer

In [ ]:
model, tokenizer = load_model_tokenizer()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-large and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Tokenization with Few_shot prompting**

In [ ]:
def tokenize_function_few(examples):

    few_shot_prompt = [
        f"""

    Example 1:
    Text: "I hate all people from country X."
    Label: Hate Speech

    Example 2:
    Text: "I love everyone, no matter where they are from."
    Label: Non-Hate Speech

    Example 3:
    Text: "People who support this political party are all idiots."
    Label: Hate Speech

    Example 4:
    Text: "Everyone has the right to be respected."
    Label: Non-Hate Speech

    Now, classify the following text:
    Text: {text}
    Label:

    """
    for text in examples["Content"]
    ]

    # This returns a dictionary of lists — what Hugging Face expects
    tokenized = tokenizer(
        few_shot_prompt,
        padding="max_length",
        truncation=True,
        max_length=128,
    )

    return tokenized


# Apply tokenization
eval_dataset_before = eval_dataset.map(tokenize_function_few, batched=True)

# Set format for PyTorch
eval_dataset_before.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

**Saving base_model**

In [ ]:
def save_model():

    output_dir = "./gpt2_hate_speech_pre"
    os.makedirs(output_dir, exist_ok=True)
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    return output_dir

output_dir_before = save_model()

**Metrics function**

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

**Few_shot Evaluation**

In [ ]:
base_model = GPT2ForSequenceClassification.from_pretrained(output_dir_before, num_labels=2)

# Tokenizer setup
tokenizer = GPT2Tokenizer.from_pretrained(output_dir_before)
tokenizer.pad_token = tokenizer.eos_token

training_args = TrainingArguments(

    output_dir="./results",
    per_device_eval_batch_size=8,
    eval_strategy="epoch",  # Evaluate every epoch
    logging_dir="./logs",
)

# Create the Trainer for the pre-fine-tuned model
trainer = Trainer(
    model = base_model,  # Model before fine-tuning
    args = training_args,
    eval_dataset = eval_dataset_before,
    compute_metrics = compute_metrics,  # Your metrics function (accuracy)
)

# Evaluate the pre-fine-tuned model
results_before = trainer.evaluate()
print("Pre-Fine-Tuning Results:", results_before)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: amirfasil09 (amirfasil09-addis-ababa-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Pre-Fine-Tuning Results: {'eval_loss': 0.9198812246322632, 'eval_model_preparation_time': 0.006, 'eval_accuracy': 0.5, 'eval_runtime': 25.1998, 'eval_samples_per_second': 19.841, 'eval_steps_per_second': 2.5}


**Setting up LoRa**

In [ ]:
# LoRA configuration
peft_config = LoraConfig(
    r=8,  # Rank
    lora_alpha=16,
    target_modules=["c_attn"],  # Targeting attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()  # Should show only ~0.1-1% of parameters are trainable

# Enable gradient checkpointing to save memory
model.gradient_checkpointing_enable()

trainable params: 1,477,120 || all params: 775,509,760 || trainable%: 0.1905


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


**Tokenization with zero_shot prompting**

In [ ]:
def tokenize_function_zero(examples):
    zero_shot_prompts = [
        f"Classify this text strictly as either 'hate_speech' or 'not_hate_speech': {text}"
        for text in examples["Content"]
    ]

    # This returns a dictionary of lists
    tokenized = tokenizer(
        zero_shot_prompts,
        padding="max_length",
        truncation=True,
        max_length=128,
    )

    return tokenized


# Apply tokenization
train_dataset = train_dataset.map(tokenize_function_zero, batched=True)
eval_dataset_after = eval_dataset.map(tokenize_function_zero, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
eval_dataset_after.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

**Fine-tuning**

In [ ]:
from transformers import TrainingArguments, Trainer

model.config.label_names = ["labels"]

training_args = TrainingArguments(
    output_dir="./gpt2_hate_speech",
    eval_strategy="epoch",  # Corrected from 'eval_strategy' (older versions)
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,  # Reduced from 8 for stability
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    gradient_accumulation_steps=4,  # Increased to compensate for smaller batch size
    logging_steps=50,
    report_to="none",
    optim="adamw_torch_fused",
    # Add these to stabilize FP16 training
    gradient_checkpointing=True,  # Saves memory
    fp16_full_eval=False,
)

from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset_after,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    data_collator=data_collator,  # Added for proper batching
)

# Verify label names
print("Label names:", model.config.label_names)  # Should output: ['labels']

# Start training
trainer.train()

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Label names: ['labels']


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.705700,0.693313,0.586000
2,0.692800,0.678912,0.586000
3,0.677500,0.674110,0.600000


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


TrainOutput(global_step=375, training_loss=0.6919013671875, metrics={'train_runtime': 193.1135, 'train_samples_per_second': 31.07, 'train_steps_per_second': 1.942, 'total_flos': 3271081328640000.0, 'train_loss': 0.6919013671875, 'epoch': 3.0})

**Saving the Fine-tunned model**

In [ ]:
def save_model():

    output_dir = "./gpt2_hate_speech_after"
    os.makedirs(output_dir, exist_ok=True)
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    return output_dir

output_dir_after = save_model()

**Evaluation Check**

In [ ]:
base_model = GPT2ForSequenceClassification.from_pretrained(output_dir_after, num_labels=2)

# Tokenizer setup
tokenizer = GPT2Tokenizer.from_pretrained(output_dir_after)
tokenizer.pad_token = tokenizer.eos_token

tunned_model = PeftModel.from_pretrained(base_model, output_dir_after)


eval_args = TrainingArguments(

    output_dir="./results",
    per_device_eval_batch_size=1,
    eval_strategy="epoch",  # Evaluate every epoch
    logging_dir="./logs",
)

# Create the Trainer for the pre-fine-tuned model
trainer = Trainer(
    model = tunned_model,  # Model before fine-tuning
    args = eval_args,
    eval_dataset = eval_dataset_after,
    compute_metrics = compute_metrics,  # Your metrics function (accuracy)
)

# Evaluate the pre-fine-tuned model
results_before = trainer.evaluate()
print("Pre-Fine-Tuning Results:", results_before)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-large and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Pre-Fine-Tuning Results: {'eval_loss': 0.6752246022224426, 'eval_model_preparation_time': 0.0095, 'eval_accuracy': 0.57, 'eval_runtime': 25.9881, 'eval_samples_per_second': 19.24, 'eval_steps_per_second': 19.24}
